In [22]:
import os
from pathlib import Path
import zipfile
import urllib
import opendatasets as od
import pandas as pd
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression

Traceback (most recent call last):
  File "/home/mgodniak/.cache/pypoetry/virtualenvs/asi-fraud-detection-BrKZcF8O-py3.10/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/home/mgodniak/.cache/pypoetry/virtualenvs/asi-fraud-detection-BrKZcF8O-py3.10/lib/python3.10/site-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/home/mgodniak/.cache/pypoetry/virtualenvs/asi-fraud-detection-BrKZcF8O-py3.10/lib/python3.10/site-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /home/mgodniak/.kaggle. Or use the environment method.


In [20]:
!kaggle datasets download -d titanic

not enough values to unpack (expected 3, got 2)


In [26]:
path = 'data/credit-data.csv'
isExisting = os.path.exists(path)

if not isExisting:
    od.download(
        "https://www.kaggle.com/datasets/kelvinkelue/credit-card-fraud-prediction/download?datasetVersionNumber=1")
    Path("data/").mkdir(parents=True, exist_ok=True)
    !mv "credit-card-fraud-prediction/fraud test.csv" "data/credit-data.csv"
    !rm "credit-card-fraud-prediction"

In [27]:
file= 'data/credit-data.csv'
fraud_data = pd.read_csv(file)

FileNotFoundError: [Errno 2] No such file or directory: 'credit-data.csv'

In [ ]:

#Usuwamy kolumne z liczba porzadkowa
fraud_data.drop('Unnamed: 0', axis=1, inplace=True)

#Tu printujemy analizę danych, przyda się przy opracowywaniu modelu

#Zwracamy typy danych dla każdej kolumny, przydatne przy przygotowywaniu danych
print("----------------------Typy danych")
print(fraud_data.dtypes) 

#Zwraca statystyki dla kolumny numerycznych jak średnia mediana itp, to może się przydać do oczyszczania danych numerycznych
print("----------------------Statystyki dla kolumn numerycznych")
print(fraud_data.describe()) 

#Tu przekształcamy kolumny z typów ogólnych na takie, które mogą być użyte przy uczeniu

#konwertuję datę najpierw na datetime, a potem datetime przekształcam na typ numeryczny
columns_trans_date = pd.to_numeric(pd.to_datetime(fraud_data['trans_date_trans_time']))

## TODO: Trzeba zrobić tak dla reszty kolumn, które na ten moment są dropnięte, część pewnie trzeba podzielić na kategoryczne, część, zgrupować, a części może faktycznie się pozbyć 


#Usuwamy niepotrzebne już kolmuny (lub wywołujące teraz błędy)
fraud_data.drop('trans_date_trans_time', axis=1, inplace=True)
fraud_data.drop('merchant', axis=1, inplace=True)
fraud_data.drop('category', axis=1, inplace=True)
fraud_data.drop('first', axis=1, inplace=True)
fraud_data.drop('last', axis=1, inplace=True)
fraud_data.drop('gender', axis=1, inplace=True)
fraud_data.drop('street', axis=1, inplace=True)
fraud_data.drop('city', axis=1, inplace=True)
fraud_data.drop('state', axis=1, inplace=True)
fraud_data.drop('job', axis=1, inplace=True)
fraud_data.drop('dob', axis=1, inplace=True)
fraud_data.drop('trans_num', axis=1, inplace=True)

#dodajemy przetworzone kolumny
fraud_data = pd.concat([fraud_data,columns_trans_date], axis=1)

toPredict = fraud_data['is_fraud']
fraud_data.drop('is_fraud', axis=1, inplace=True)

#Definiujemy użyty model, w tym wypadku regresja logistyczna
model = LogisticRegression(max_iter=1000)

#To teoretycznie oblicza wyniki uczenia, jako estymator daję model, dane i zmienną do przewidzenia, cv to definicja że ma być użyte 10-fold, oraz wynik na jakim się skupiamy to dokładność
## TODO chociaż coś tu jest ewidentnie nie tak, wywaliłem prawie wszystko i mam 99% dokładności xD Nie wiem o co tu chodzi ale trzeba sprawdzić
scores = cross_val_score(model, fraud_data, toPredict, cv=10, scoring='accuracy')
print(scores)